## Importação das Bibliotecas necessárias
A biblioteca copy foi usada para realizar uma cópia de valores ao longo do algoritmo de pré-processamento

In [ ]:
import pandas as pd
import numpy as np
import operator
from copy import deepcopy

## Leitura dos Arquivos

In [ ]:
music_data = pd.read_csv("music_data.csv")
hits = pd.read_csv("hits.csv")
target = pd.read_csv("target.csv")

## Inicialização das variáveis de controle
Estas variáveis serão usadas ao longo do código para facilitar a legibilidade

In [ ]:
music_id = 0
music = 1
music_value = 4
usr_id = 0
usr_musics = 1
topHits = 100  # decidi por utilizar 100 top hits para a recomendação
numUsers = len(hits.groupby(['user_id']).count())-1

# Algoritmo de Pré-Processamento

É necessário um algoritmo de pré processamento para deixar os dados do jeito correto estabelecido para o funcionamento do algoritmo.
Decidi usar a seguinte abordagem: Terei uma matriz com todos os usuários. Cada usuário será uma linha da matriz. Esta matriz (chamada users) tem duas colunas: a primeira para o id do usuário naquela linha, e a segunda que é outra matriz (chamada listaMusicas). Esta contém, na primeira coluna, os 100 hits mais tocados, e na segunda coluna, a quantidade que aquele determinado usuário ouviu àquela música.
O algoritmo de pré-processamento não se preocupa com o caso de usuários novos que ainda não ouviram a nenhuma música, pois o objetivo era recomendar apenas para usuários target já pré-definidos que já haviam ouvido alguma música.


## Inicialização das matrizes usadas pelo algoritmo de pré-processamento

In [ ]:
musics = [[]for i in range(topHits)]
# modelo do vetor user = [0,[[105301,69],[51,0],...]]
users = [[]for i in range(numUsers)]

# pegando as top 100 musicas, baseado em numero de plays
music_data.sort_values(['plays'], ascending=False, inplace=True)
top_musics = np.array(music_data['music_id'][:topHits])

# fazendo o vetor base de musicas com todos os valores 0
for i in range(len(top_musics)):
    musics[i].append(top_musics[i])
    musics[i].append(0)

## Atribuição dos valores ouvidos de cada top_hit por cada usuário
A matriz de cada usuário possui vários valores 0, pois dificilmente um usuário terá escutado todas as músicas do top 100. Como o vetor base deixa como padrão o valor 0 para todas as músicas, a parte abaixo é encarregada de sobrescrever estes valores 0 nas músicas que o usuário já ouviu, com o valor correto, e fazer isto para todos os usuários da base de dados.

In [ ]:
linha = 0
for j in range(numUsers):
    users[j] = [hits.iloc[linha, usr_id], musics]  # users[j] = [user_id, vetor base com os top 100 hits]
    if linha < len(hits):  # if usado para garantir que na última iteração não ocorra um index out of bounds
        listaMusicas = deepcopy(musics)  # deepcopy para que apenas os valores de musics sejam transferidos para listaMusicas. Particularidade da linguagem Python.
        while hits.iloc[linha, usr_id] == j:  # hits tem várias linhas de informação para cada usuário. Enquanto eu estiver no mesmo usuário faça:
            musica = hits.iloc[linha, music]
            for i in range(len(top_musics)):
                if users[j][usr_musics][i][music_id] == musica:  # se a musica atual existir no vetor do usuario
                    listaMusicas[i][music] = hits.iloc[linha][music_value]
            linha += 1
        users[j] = [hits.iloc[linha, usr_id], listaMusicas]